In [ ]:
def Mindspore_ansatz(Structure_p:np.array,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    Structure_p:np.array DQAS中的权重参数,
    Ansatz_p:np.array  DQAS中的Ansatz参数,
    
    """
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    
    prg = PRGenerator('encoder')
    nqbits = n_qbits
    encoder = Circuit()
    encoder += UN(H, nqbits)                                 
    for i in range(nqbits):                                  
        encoder += RY(prg.new()).on(i)                 
        
    ansatz = Circuit()
    pr_gen = PRGenerator('ansatz')
    #print(my_stp.shape)
    for i in range(n_layer):
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):
            #print(my_stp[i,index_op])
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=pr_gen.new()),time=float(my_stp[i,index_op])).circuit
            paramertized_part_count+=1
            
        # for index_op,each_op in enumerate(bound_opeartor_pool):
        #     op = GroupedPauli(each_op)
        #     tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
        #     matrix = tmp_cir.matrix()
        #     ansatz += UnivMathGate(matrix_value=matrix*float(my_stp[i,index_op+paramertized_part_count]),name=f'{my_stp[i,index_op+paramertized_part_count]}*{op.pauli_string}').on(range(n_qbits))  
    
    finnal_ansatz = encoder.as_encoder() + ansatz.as_ansatz()
    return finnal_ansatz